### [TensorFlow] Lec-04 Multi variable linear regression
이전복습내용)
- Hypothesis : 가설함수 , 우리의 예측 H(x)=Wx +b
- Cost function : 비용함수, 우리의 예측과 실제 값의 차이, cost(W) = (가설-실제데이터)제곱의 평균
- Gradient Descent : 경사하강알고리즘, cost 를 최소화하는 w를 찾아가는 방법,  w=w-(learning rate) X (비용미분)

1) 변수가 하나일 때 가설함수 Wx + b
2) 변수가 여러개  Multi variable,늘어난 변수만큼 가중치를 필요하게 된다. w1x1 + w2x2 + w3x3 + b
3) 몇 백개의 변수 -> Matrix Multiplication 이용
4) 앞 matrix의 열의 개수와 뒤의 matrix의 행의 개수가 일치해야 한다.


In [ ]:
data = np.array([
    [73., 80., 75., 152.],
    [93., 88., 93., 185.],
    [89., 91., 90., 180.],
    [96., 98., 100., 196.],
    [73., 66., 70., 142.]
], dtype= np.float32)

x = data[:, :-1]
y= data [:, [-1]]

w= tf.Variable(tf.random_normal([3,1]))
b= tf.Variable(tf.random_normal([1]))

learning_rate = 0.000001


def predict(x):
    return tf.matmul(x,w) + b

n_epochs = 2000
for i in range(n_epochs+1):
    with tf.GradientTape() as tape:
        cost = tf.reduce_mean((tf.square(predict(x)-y)))
        w_grad, b_grad = tape.gradient(cost, [w,b])
        
        w.assign_sub(learning_rate * w_grad)
        b.assign_sub(learning_rate * b_grad)
        
        if i % 100 == 0:
            print("{:5}|{:10.4f}".format(i, cost.numpy()))
        

### [TensorFlow] Lab-05-1 Logistic Regression
1) What is Logistic Regression?
- classification (0:positive / 1:negative) 
- Logistic VS Linear : Discrete(데이터 분리 구별이 가능, 0과 1로 분류 가능) / Continous (시간같은 연속적인 데이터, 수치형 )

2) How to Solve?
- Hypothesis Representation : 직선 -> Pass/Fail 구부러지는 그래프
- Linear(wx+b) - Logistic(0< a <1) - DecisionBoundary( a>0.5 0.5보다 크면1 작으면 0)
- Sigmoid/Logisitic Function : Linear를 통해 나온 수치화 값을 Logistic function을 통해 sigmoid 함수 
- Decision Boundary 
- Cost Function : 랜덤하게 만들어진 W 값을 최적의 파라미터로 만들어주는 것,   
- Optimizer 

### Lab 05 Logistic Classification (diabetes) - Eager Execution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
tf.random.set_seed(777)  # for reproducibility
print(tf.__version__)

In [ ]:
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

print(x_train.shape, y_train.shape)
print(xy)


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))


In [ ]:
W = tf.Variable(tf.random.normal((8, 1)), name='weight')
b = tf.Variable(tf.random.normal((1,)), name='bias')

In [ ]:
def logistic_regression(features):
    hypothesis  = tf.divide(1., 1. + tf.exp(tf.matmul(features, W) + b))
    return hypothesis

In [ ]:

def loss_fn(hypothesis, features, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(logistic_regression(features)) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [ ]:

def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

In [ ]:
def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(logistic_regression(features),features,labels)
    return tape.gradient(loss_value, [W,b])

In [ ]:
EPOCHS = 1001

for step in range(EPOCHS):
    for features, labels  in iter(dataset):
        grads = grad(logistic_regression(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
        if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),features,labels)))